### Please install the required Python modules/SDKs

In [ ]:
!pip install azure-ai-formrecognizer

import sys

### Importing Azure Form Recognizer Python modules

In [44]:
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.formrecognizer import FormTrainingClient
from azure.core.credentials import AzureKeyCredential

### Please use this notebook to write your own solution for this exercise. 

### Enter Form Recognizer endpoint and key and instantiate object

In [45]:
AZURE_FORM_RECOGNIZER_ENDPOINT = "ENDPOINT"
AZURE_FORM_RECOGNIZER_KEY = "KEY"

In [46]:
endpoint = AZURE_FORM_RECOGNIZER_ENDPOINT
key = AZURE_FORM_RECOGNIZER_KEY

In [47]:
form_training_client = FormTrainingClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [48]:
saved_model_list = form_training_client.list_custom_models()

### Provide training source data URL

In [49]:
trainingDataUrl = "TRAINING_URL"

### Perform labeled traning
* Use_training_labels=True
* You need at least 5 labeled documents with the `ocr.json` and `labels.json` files; otherwise, you will run into errors.

In [ ]:
# Fill in the code to begin labeled training
training_process = form_training_client.begin_training(trainingDataUrl, use_training_labels=True)
custom_model = training_process.result()

### Get model info

In [ ]:
custom_model

In [ ]:
# Fill in the code to get model ID
custom_model.model_id

In [ ]:
# Fill in the code to get status
custom_model.status

In [ ]:
custom_model.training_started_on

In [ ]:
custom_model.training_completed_on

In [ ]:
# Fill in the code to get training documents
custom_model.training_documents

In [ ]:
for doc in custom_model.training_documents:
    print("Document name: {}".format(doc.name))
    print("Document status: {}".format(doc.status))
    print("Document page count: {}".format(doc.page_count))
    print("Document errors: {}".format(doc.errors))

In [ ]:
custom_model.properties

In [ ]:
custom_model.submodels

In [ ]:
for submodel in custom_model.submodels:
    print(
        "The submodel with form type '{}' has recognized the following fields: {}".format(
            submodel.form_type,
            ", ".join(
                [
                    field.label if field.label else name
                    for name, field in submodel.fields.items()
                ]
            ),
        )
    )

In [ ]:
custom_model.model_id

In [ ]:
custom_model_info = form_training_client.get_custom_model(model_id=custom_model.model_id)
print("Model ID: {}".format(custom_model_info.model_id))
print("Status: {}".format(custom_model_info.status))
print("Training started on: {}".format(custom_model_info.training_started_on))
print("Training completed on: {}".format(custom_model_info.training_completed_on))

### Perform prediction
* Please download and save one of the PDFs not used in training, and then upload it to your Azure blob container.
* After that, please create an Azure SAS URL with only READ access. Use the URL below to predict test/prediction with the model.

In [ ]:
new_test_url = "TEST_URL"

In [ ]:
new_test_url

In [ ]:
form_recognizer_client = FormRecognizerClient(endpoint=endpoint, credential=AzureKeyCredential(key))

In [ ]:
custom_model.model_id

In [ ]:
custom_model_info.model_id

In [ ]:
# Fill in code to begin prediction
custom_test_action = form_recognizer_client.begin_recognize_custom_forms_from_url(model_id=custom_model_info.model_id, form_url=new_test_url)

In [ ]:
custom_test_action.result

In [ ]:
custom_test_action.status()

In [ ]:
custom_test_action_result = custom_test_action.result()

In [ ]:
for recognized_content in custom_test_action_result:
    print("Form type: {}".format(recognized_content.form_type))
    for name, field in recognized_content.fields.items():
        print("Field '{}' has label '{}' with value '{}' and a confidence score of {}".format(
            name,
            field.label_data.text if field.label_data else name,
            field.value,
            field.confidence
        ))